In [2]:
pip install fiftyone

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 943.0/943.0 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.9 MB/s eta 0:00:0

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("robinreni/revitsone-5class")

print("Path to dataset files:", path)

100%|██████████| 983M/983M [00:15<00:00, 65.6MB/s]


Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/robinreni/revitsone-5class/versions/1


In [21]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
!pwd

/content


In [29]:
!unzip /content/drive/MyDrive/driving_data.zip -d /content/driving_data

Streaming output truncated to the last 5000 lines.
  inflating: /content/driving_data/revitsone-5classes/Revitsone-5classes/talking_phone/img_37249.jpg  
  inflating: /content/driving_data/revitsone-5classes/Revitsone-5classes/talking_phone/img_37255.jpg  
  inflating: /content/driving_data/revitsone-5classes/Revitsone-5classes/talking_phone/img_37258.jpg  
  inflating: /content/driving_data/revitsone-5classes/Revitsone-5classes/talking_phone/img_37265.jpg  
  inflating: /content/driving_data/revitsone-5classes/Revitsone-5classes/talking_phone/img_37269.jpg  
  inflating: /content/driving_data/revitsone-5classes/Revitsone-5classes/talking_phone/img_37290.jpg  
  inflating: /content/driving_data/revitsone-5classes/Revitsone-5classes/talking_phone/img_37291.jpg  
  inflating: /content/driving_data/revitsone-5classes/Revitsone-5classes/talking_phone/img_37307.jpg  
  inflating: /content/driving_data/revitsone-5classes/Revitsone-5classes/talking_phone/img_37310.jpg  
  inflating: /content/

In [35]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image

# Define dataset path
dataset_path = "/content/driving_data/Revitsone-5classes"

# Image parameters
IMG_SIZE = (224, 224)  # Resize all images to 224x224
BATCH_SIZE = 64

# Function to check and remove corrupt images
def remove_corrupt_images(dataset_path):
    corrupt_files = []
    for root, _, files in os.walk(dataset_path):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                img = Image.open(file_path)
                img.verify()  # Verify image integrity
            except (IOError, SyntaxError):
                corrupt_files.append(file_path)
                os.remove(file_path)
                print(f"🗑️ Removed corrupt image: {file_path}")

    print(f"✅ Cleaning done! {len(corrupt_files)} corrupt images removed.")

# Run corrupt image removal
remove_corrupt_images(dataset_path)

# Data augmentation and preprocessing
datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values
    validation_split=0.2,  # Split dataset into train and validation
    horizontal_flip=True,
    zoom_range=0.2
)

# Load training data
train_data = datagen.flow_from_directory(
    dataset_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',  # Ensure it matches the 5-class output
    subset='training'
)

# Load validation data
val_data = datagen.flow_from_directory(
    dataset_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',  # Ensure it matches the 5-class output
    subset='validation'
)

# Display class indices
print("✅ Dataset successfully loaded.")
print("Class indices:", train_data.class_indices)
print("Number of classes detected:", len(train_data.class_indices))


🗑️ Removed corrupt image: /content/driving_data/Revitsone-5classes/turning/img_101434.jpg
🗑️ Removed corrupt image: /content/driving_data/Revitsone-5classes/turning/img_70552.jpg
🗑️ Removed corrupt image: /content/driving_data/Revitsone-5classes/turning/img_67523.jpg
🗑️ Removed corrupt image: /content/driving_data/Revitsone-5classes/turning/img_84605.jpg
🗑️ Removed corrupt image: /content/driving_data/Revitsone-5classes/turning/img_62337.jpg
🗑️ Removed corrupt image: /content/driving_data/Revitsone-5classes/turning/img_8771.jpg
🗑️ Removed corrupt image: /content/driving_data/Revitsone-5classes/other_activities/img_22266.jpg
🗑️ Removed corrupt image: /content/driving_data/Revitsone-5classes/other_activities/img_20398.jpg
🗑️ Removed corrupt image: /content/driving_data/Revitsone-5classes/other_activities/img_13541.jpg
🗑️ Removed corrupt image: /content/driving_data/Revitsone-5classes/other_activities/img_79.jpg
🗑️ Removed corrupt image: /content/driving_data/Revitsone-5classes/other_acti

In [36]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model

# Load the pre-trained MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
base_model.trainable = False

# Add custom classification layers
x = Flatten()(base_model.output)
x = Dropout(0.5)(x)
x = Dense(5, activation='softmax')(x)  # 5 classes in the dataset

# Create the model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1 (Conv2D)            │ (None, 112, 112, 32)   │            864 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn_Conv1                  │ (None, 112, 112, 32)   │            128 │ Conv1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1_relu (ReLU)         │ (None, 112, 112, 32)   │              0 │ bn_Conv1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise   │ (None, 112, 112, 32)   │            288 │ Conv1_relu[0][0]       │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 112, 112, 32)   │            128 │ expanded_conv_depthwi… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 112, 112, 32)   │              0 │ expanded_conv_depthwi… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project     │ (None, 112, 112, 16)   │            512 │ expanded_conv_depthwi… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project_BN  │ (None, 112, 112, 16)   │             64 │ expanded_conv_project… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand (Conv2D)   │ (None, 112, 112, 96)   │          1,536 │ expanded_conv_project… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_BN         │ (None, 112, 112, 96)   │            384 │ block_1_expand[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_relu       │ (None, 112, 112, 96)   │              0 │ block_1_expand_BN[0][… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_pad               │ (None, 113, 113, 96)   │              0 │ block_1_expand_relu[0… │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise         │ (None, 56, 56, 96)     │            864 │ block_1_pad[0][0]      │
│ (DepthwiseConv2D)         │                        │                │                        │
├──────────────────────

 Total params: 2,571,589 (9.81 MB)

 Trainable params: 313,605 (1.20 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [37]:
# Train the model
model.fit(train_data, validation_data=val_data, epochs=10)


Epoch 1/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 169s 1s/step - accuracy: 0.5644 - loss: 5.9224 - val_accuracy: 0.8463 - val_loss: 1.5748
Epoch 2/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 183s 1s/step - accuracy: 0.8647 - loss: 0.7916 - val_accuracy: 0.8151 - val_loss: 2.1427
Epoch 3/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 155s 1s/step - accuracy: 0.8756 - loss: 0.8140 - val_accuracy: 0.8430 - val_loss: 2.0310
Epoch 4/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 190s 1s/step - accuracy: 0.9079 - loss: 0.6819 - val_accuracy: 0.8798 - val_loss: 2.0877
Epoch 5/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 143s 1s/step - accuracy: 0.9223 - loss: 0.5925 - val_accuracy: 0.8859 - val_loss: 1.9777
Epoch 6/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 146s 1s/step - accuracy: 0.9132 - loss: 0.7210 - val_accuracy: 0.8836 - val_loss: 2.6648
Epoch 7/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - accuracy: 0.9325 - loss: 0.5726 - val_accuracy: 0.8738 - val_loss: 3.2487
Epoch 8/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 144s 1s/step - accuracy: 0.9332 - loss: 0.6563 - val_accu